In [24]:
import polars as pl
import re        

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

In [2]:
%%time
cats = pl.read_csv("category_tree.csv"
           ).with_columns(pl.when(pl.col("parent_id").is_null()).then(pl.lit(0)).otherwise(-99).cast(pl.Int8).alias("level"),
                          pl.col("parent_id").cast( pl.Int16),
                          pl.col("cat_id").cast( pl.Int16),
                          #pl.lit(0).cast( pl.Int8).alias("end"),
                          #pl.lit(0).cast( pl.Int8).alias("population_cats"),
                          #pl.lit(0).cast( pl.Int8).alias("population_names"),
                         )#.with_columns( pl.when(pl.col("parent_id").is_null()).then(pl.lit(0)).otherwise(-1) )
cats = cats.join(cats[["cat_id","level"]],left_on="parent_id",right_on="cat_id",how="left")
cats = cats.with_columns( pl.when(pl.col("level")>=0).then(pl.col("level")).otherwise(pl.col("level_right")+1) ).drop("level_right")
cats = cats.join(cats[["cat_id","level"]],left_on="parent_id",right_on="cat_id",how="left")
cats = cats.with_columns( pl.when(pl.col("level")>=0).then(pl.col("level")).otherwise(pl.col("level_right")+1) ).drop("level_right")
cats = cats.join(cats[["cat_id","level"]],left_on="parent_id",right_on="cat_id",how="left")
cats = cats.with_columns( pl.when(pl.col("level")>=0).then(pl.col("level")).otherwise(pl.col("level_right")+1) ).drop("level_right")
cats = cats.join(cats[["cat_id","level"]],left_on="parent_id",right_on="cat_id",how="left")
cats = cats.with_columns( pl.when(pl.col("level")>=0).then(pl.col("level")).otherwise(pl.col("level_right")+1) ).drop("level_right")

cats = cats.with_columns( pl.when(pl.col("level")==4).then(pl.col("cat_id")).alias("p4") )

cats = cats.with_columns( pl.when(pl.col("level")==4).then(pl.col("parent_id")).alias("p3") )
cats = cats.with_columns( pl.when(pl.col("level")==3).then(pl.col("cat_id")).otherwise(pl.col("p3")).alias("p3") )

cats = cats.join(cats[["cat_id","parent_id"]],left_on="p3",right_on="cat_id",how="left").rename({"parent_id_right":"p2"})
cats = cats.with_columns( pl.when(pl.col("level")==2).then(pl.col("cat_id")).otherwise(pl.col("p2")).alias("p2") )

cats = cats.join(cats[["cat_id","parent_id"]],left_on="p2",right_on="cat_id",how="left").rename({"parent_id_right":"p1"})
cats = cats.with_columns( pl.when(pl.col("level")==1).then(pl.col("cat_id")).otherwise(pl.col("p1")).alias("p1") )

cats = cats.join(cats[["cat_id","parent_id"]],left_on="p1",right_on="cat_id",how="left").rename({"parent_id_right":"p0"})
cats = cats.with_columns( pl.when(pl.col("level")==0).then(pl.col("cat_id")).otherwise(pl.col("p0")).alias("p0") )

cats

CPU times: user 50.7 ms, sys: 18.8 ms, total: 69.6 ms
Wall time: 134 ms


cat_id,parent_id,cat_name,level,p4,p3,p2,p1,p0
i16,i16,str,i8,i16,i16,i16,i16,i16
1,null,"""Электроника""",0,null,null,null,null,1
2,null,"""Ноутбуки и компьютеры""",0,null,null,null,null,2
3,null,"""Бытовая техника""",0,null,null,null,null,3
4,null,"""Детские товары""",0,null,null,null,null,4
5,null,"""Дача, сад и огород""",0,null,null,null,null,5
…,…,…,…,…,…,…,…,…
31426,14355,"""Аккумуляторы для мобильных тел…",4,31426,14355,1005,101,1
31431,14355,"""Коннекторы и контейнеры для те…",4,31431,14355,1005,101,1
31433,14355,"""Корпусные детали для телефонов""",4,31433,14355,1005,101,1


In [3]:
#population_cats	end	population_trains

#cats = 
cats0 = pl.read_csv("category_tree.csv"
           ).with_columns(pl.when(pl.col("parent_id").is_null()).then(pl.lit(0)).otherwise(-99).cast(pl.Int8).alias("level"),
                          pl.col("parent_id").cast( pl.Int16),
                          pl.col("cat_id").cast( pl.Int16),
                          pl.lit(0).cast( pl.Int8).alias("end"),
                          #pl.lit(0).cast( pl.Int8).alias("population_cats"),
                          #pl.lit(0).cast( pl.Int8).alias("population_names"),
                         )#.with_columns( pl.when(pl.col("parent_id").is_null()).then(pl.lit(0)).otherwise(-1) )
cats0=cats0.join(cats[["cat_id","parent_id"]],left_on="cat_id",right_on="parent_id",how="left"
         ).rename({"cat_id_right":"population_cats"}
                 ).group_by("cat_id"
                           ).agg(pl.col("population_cats").count().cast(pl.Int8)
                                ).with_columns(pl.when(pl.col("population_cats")==0
                                                      ).then(pl.lit(1)
                                                            ).otherwise(pl.lit(0)
                                                                       ).cast( pl.Int8).alias("end")
                                              )
cats0
cats1=cats0.select("cat_id"
                  ).join(pl.read_parquet("labeled_train.parquet"
                                ).group_by("cat_id").len(
                                        ).with_columns(pl.col("cat_id").cast(pl.Int16)),
                 on="cat_id",
                 how="left"
                ).rename({"len":"population_trains"}
                        ).fill_null(0)


cats=cats.join(cats0,on="cat_id",how="left").join(cats1,on="cat_id",how="left")
cats

cat_id,parent_id,cat_name,level,p4,p3,p2,p1,p0,population_cats,end,population_trains
i16,i16,str,i8,i16,i16,i16,i16,i16,i8,i8,u32
1,null,"""Электроника""",0,null,null,null,null,1,18,0,540
2,null,"""Ноутбуки и компьютеры""",0,null,null,null,null,2,15,0,20
3,null,"""Бытовая техника""",0,null,null,null,null,3,7,0,0
4,null,"""Детские товары""",0,null,null,null,null,4,11,0,0
5,null,"""Дача, сад и огород""",0,null,null,null,null,5,9,0,0
…,…,…,…,…,…,…,…,…,…,…,…
31426,14355,"""Аккумуляторы для мобильных тел…",4,31426,14355,1005,101,1,0,1,0
31431,14355,"""Коннекторы и контейнеры для те…",4,31431,14355,1005,101,1,0,1,0
31433,14355,"""Корпусные детали для телефонов""",4,31433,14355,1005,101,1,0,1,0


In [4]:
#population_trains_full
labeled=pl.read_parquet("labeled_train.parquet"
               ).with_columns(pl.col("cat_id").cast(pl.Int16))
ps=[]                              
for level in range(5):                            
    ps.append(labeled.join(cats,on="cat_id",how="left"
                          ).group_by(f"p{level}").len().rename({f"p{level}":"cat_id","len":"population_trains_full"})#.sort("p0")
             )
cats=cats.join( cats.select("cat_id").join(pl.concat(ps),on="cat_id",how="left").fill_null(0),
          on="cat_id",how="left")
cats

cat_id,parent_id,cat_name,level,p4,p3,p2,p1,p0,population_cats,end,population_trains,population_trains_full
i16,i16,str,i8,i16,i16,i16,i16,i16,i8,i8,u32,u32
1,null,"""Электроника""",0,null,null,null,null,1,18,0,540,307950
2,null,"""Ноутбуки и компьютеры""",0,null,null,null,null,2,15,0,20,180160
3,null,"""Бытовая техника""",0,null,null,null,null,3,7,0,0,145342
4,null,"""Детские товары""",0,null,null,null,null,4,11,0,0,6583
5,null,"""Дача, сад и огород""",0,null,null,null,null,5,9,0,0,805
…,…,…,…,…,…,…,…,…,…,…,…,…
31426,14355,"""Аккумуляторы для мобильных тел…",4,31426,14355,1005,101,1,0,1,0,0
31431,14355,"""Коннекторы и контейнеры для те…",4,31431,14355,1005,101,1,0,1,0,0
31433,14355,"""Корпусные детали для телефонов""",4,31433,14355,1005,101,1,0,1,0,0


In [5]:
cats.filter(pl.col("end")==1).filter(pl.col("level")==1).filter(pl.col("population_trains")<=3)

cat_id,parent_id,cat_name,level,p4,p3,p2,p1,p0,population_cats,end,population_trains,population_trains_full
i16,i16,str,i8,i16,i16,i16,i16,i16,i8,i8,u32,u32
117,1,"""Радиоуправляемые модели""",1,null,null,null,117,1,0,1,0,0
152,4,"""Детская одежда и обувь""",1,null,null,null,152,4,0,1,0,0
164,5,"""Парники и теплицы""",1,null,null,null,164,5,0,1,0,0
169,6,"""Макароны, крупы""",1,null,null,null,169,6,0,1,0,0
170,6,"""Растительное масло""",1,null,null,null,170,6,0,1,0,0
…,…,…,…,…,…,…,…,…,…,…,…,…
417,22,"""Школьная форма для мальчиков""",1,null,null,null,417,22,0,1,0,0
438,28,"""Игровые ноутбуки""",1,null,null,null,438,28,0,1,0,0
445,28,"""4K гейминг""",1,null,null,null,445,28,0,1,0,0


In [7]:
%%time
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

MODEL_NAME = "IlyaGusev/saiga_yandexgpt_8b"

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    load_in_8bit=True,
    torch_dtype=torch.bfloat16,
    #device_map="auto"
)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
print(generation_config)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 128009,
  "max_length": 1024000,
  "max_new_tokens": 2048,
  "pad_token_id": 0,
  "repetition_penalty": 1.1,
  "temperature": 0.6,
  "top_p": 0.95
}

CPU times: user 22.1 s, sys: 16.2 s, total: 38.3 s
Wall time: 50.3 s


In [45]:
%%time
reg1 = re.compile(r'^\d+\.')
reg2 = re.compile(r"^\d+\.\s+")

emptycats = cats.filter(pl.col("end")==1
                       ).filter(pl.col("level")==1
                               ).filter(pl.col("population_trains")<=3
                                       )
newitems=[]
for row in empties.rows(named=True):
    #print(row)
    print(row['cat_name'],row['cat_id'])
    
    #print(empties.row(index))
    #print(name)
    
    query = f"Составь пронумерованный одноуровневый список товаров категории '{row['cat_name']}'. Наименования товаров пиши в единственном числе. Комментарии к списку не нужны."
    #query = f"Составь пронумерованный одноуровневый список товаров категории '{row['cat_name']}'. Наименования товаров пиши в единственном числе. В конце списка напиши комментарий."
    prompt = tokenizer.apply_chat_template([{
        "role": "user",
        "content": query
    }], tokenize=False, add_generation_prompt=True)
    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    data = {k: v.to(model.device) for k, v in data.items()}
    data.pop("token_type_ids", None)
    output_ids = model.generate(**data, generation_config=generation_config)[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True).strip()
    #print(query)
    print(output)
    print()
    print("==============================")
    print()   
    items = list(filter(reg1.search, output.split('\n'))) #фильтр от комментариев и пустых строк
    items = list(map(lambda x:reg2.sub("",x), items))
    newitems+= [[item, row['cat_id']] for item in items]   
    #break

newitems

Радиоуправляемые модели 117
1. Радиоуправляемый самолет
2. Радиоуправляемая машина (багги, монстр-трак)
3. Радиоуправляемый катер
4. Радиоуправляемый квадрокоптер
5. Радиоуправляемый робот
6. Радиоуправляемый вертолет
7. Радиоуправляемая лодка
8. Радиоуправляемая подводная лодка
9. Радиоуправляемый танк
10. Радиоуправляемая модель поезда


Детская одежда и обувь 152
1. Детский комбинезон
2. Футболка детская
3. Джинсы детские
4. Ботинки для детей
5. Сандалии детские
6. Пижама для малышей
7. Шапка детская
8. Перчатки детские
9. Свитер детский
10. Рюкзак школьный (детский)


Парники и теплицы 164
1. Теплица арочная с покрытием из поликарбоната
2. Парник туннельный для выращивания рассады
3. Тепличный комплекс на фундаменте
4. Мобильный парник-бабочка
5. Мини-теплица для домашнего использования
6. Система автоматического полива для теплиц
7. Оцинкованная рама для тепличного комплекса
8. Вентиляционный комплект для проветривания теплиц
9. Инсектицидные сетки для защиты растений от вредителе

1. Зеркало настенное "Классика"
2. Зеркало напольное "Барокко"
3. Зеркало с подсветкой "Современный стиль"
4. Напольный туалетный столик с зеркалом "Элит"
5. Угловое зеркало "Скандинавский дизайн"
6. Потолочное зеркало "Романтический стиль"
7. Настольное зеркало "Минимализм"
8. Зеркальная панель для ванной комнаты "Акварель"
9. Зеркало-шкаф "Фэшн"
10. Декоративное зеркало "Арт"


Мебель для прихожей 284
1. Шкаф-купе
2. Тумба под обувь
3. Пуф
4. Вешалка настенная
5. Зеркало с подсветкой
6. Полка открытая
7. Коврик придверный
8. Столик приставной
9. Органайзер для мелочей
10. Подставка для зонтов


Надувная, плетеная и прочая мебель 287
1. Надувной матрас
2. Плетеное кресло-качалка
3. Воздушный шезлонг
4. Гамак с тканевыми подвесами
5. Складная пляжная кровать
6. Подвесное кресло из ротанга
7. Водяной матрац
8. Тент для отдыха на природе
9. Пуф надувной
10. Садовый диван из искусственного ротанга
11. Лодка надувная для рыбалки
12. Коврик для йоги надувной
13. Плавучий бассейн
14. Стул ск

[['Радиоуправляемый самолет', 117],
 ['Радиоуправляемая машина (багги, монстр-трак)', 117],
 ['Радиоуправляемый катер', 117],
 ['Радиоуправляемый квадрокоптер', 117],
 ['Радиоуправляемый робот', 117],
 ['Радиоуправляемый вертолет', 117],
 ['Радиоуправляемая лодка', 117],
 ['Радиоуправляемая подводная лодка', 117],
 ['Радиоуправляемый танк', 117],
 ['Радиоуправляемая модель поезда', 117],
 ['Детский комбинезон', 152],
 ['Футболка детская', 152],
 ['Джинсы детские', 152],
 ['Ботинки для детей', 152],
 ['Сандалии детские', 152],
 ['Пижама для малышей', 152],
 ['Шапка детская', 152],
 ['Перчатки детские', 152],
 ['Свитер детский', 152],
 ['Рюкзак школьный (детский)', 152],
 ['Теплица арочная с покрытием из поликарбоната', 164],
 ['Парник туннельный для выращивания рассады', 164],
 ['Тепличный комплекс на фундаменте', 164],
 ['Мобильный парник-бабочка', 164],
 ['Мини-теплица для домашнего использования', 164],
 ['Система автоматического полива для теплиц', 164],
 ['Оцинкованная рама для теп

In [53]:
schema={"item_name": pl.String,"cat_id": pl.Int16}
pl.DataFrame(newitems,
                  schema=schema, orient="row").write_parquet("dirty new items.parquet")
pl.DataFrame(newitems,
                  schema=schema, orient="row")

item_name,cat_id
str,i16
"""Радиоуправляемый самолет""",117
"""Радиоуправляемая машина (багги…",117
"""Радиоуправляемый катер""",117
"""Радиоуправляемый квадрокоптер""",117
"""Радиоуправляемый робот""",117
…,…
"""Сахарница""",469
"""Молочник""",469
"""Тостер""",469
